In [ ]:
pip install requests beautifulsoup4

In [ ]:
!pip install selenium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
--2023-11-06 19:05:37--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 209.85.147.91, 209.85.147.93, 209.85.147.190, ...
Connecting to dl.google.com (dl.google.com)|209.85.147.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103888432 (99M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>]  99.08M   251MB/s    in 0.4s    

2023-11-06 19:05:37 (251 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [103888432/103888432]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of '

In [ ]:
def countNumberOfSchools(county, district):
  url = f'https://www.ed-data.org/district/{county}/{district}'
  districtDriver = webdriver.Chrome(options=options)
  districtDriver.get(url)
  button = districtDriver.find_element(By.ID, "entity-list-btn")
  button.click()
  table = districtDriver.find_element(By.ID, "DataTables_Table_0")
  tableBody = table.find_element(By.TAG_NAME, "tbody")
  schools = tableBody.find_elements(By.TAG_NAME, "tr")
  return len(schools)

In [ ]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select
from google.colab import drive
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from IPython.display import clear_output

#https://blog.devgenius.io/use-selenium-webdriver-in-google-colab-d5f2dba1d9f5


# Variables for the website URL
county = 'Alameda'

countyUrl = f'https://www.ed-data.org/county/{county}'
districts = []
countyResponse = requests.get(countyUrl)


# Check if the request was successful
if countyResponse.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(countyResponse.content, 'html.parser')

    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    masterTable = pd.read_excel('/content/Master County Tracker Dataset for CA District Data - Alameda.xlsx', index_col=0).iloc[0:0, 0:19].reset_index()
    display(masterTable)


    sheet = pd.read_excel('/content/CA All School Districts Spreadsheet - Need and Equity Data.xlsx', index_col=0)
    pollutionInfoTable = sheet[["County", "District", "Average Pollution Burden ", "Average Pollution Burden Percentile"]].rename(columns={"Average Pollution Burden ":"Average Pollution Burden"})

    #get all counties
    counties = []
    option_elements = [option for option in soup.find_all('option') if option.parent.name == 'select']
    for option in option_elements[1:]:
      county = option.text.strip()
      counties.append(county)

    #for every county, find all the districts and their information
    for county in counties:
      countyUrl = f'https://www.ed-data.org/county/{county}'
      driver = webdriver.Chrome(options=options)
      driver.get(countyUrl)
      button = driver.find_element(By.ID, "entity-list-btn")
      button.click()
      table = driver.find_element(By.ID, "DataTables_Table_0")
      tableBody = table.find_element(By.TAG_NAME, "tbody")
      elementList = tableBody.find_elements(By.TAG_NAME, "tr")
      for element in elementList:
        district = element.find_elements(By.TAG_NAME, "td")[0]
        district = district.text

        url = f'https://www.ed-data.org/district/{county}/{district}'

        # Send a GET request to the website
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the div element with class "profiledata-container key-att"
        div_element = soup.find('div', class_='profiledata-container key-att')

        # Find the ul elements within the div_element
        if not div_element or "SBE" in district:
          print("District: ", county, ', ', district)
          print("---------------------")
          print("Could not find data, please manually check")
          print("\n")
          continue
        ul_elements = div_element.find_all('ul', class_='profiledata')

        # Define a dictionary to store the information
        district_info = {}

        # Loop through the ul elements to collect the information
        for ul in ul_elements:
            li_elements = ul.find_all('li')
            if len(li_elements) == 2:
                key = li_elements[0].text.strip()
                value = li_elements[1].find('span').text.strip()
                district_info[key] = value
        if district_info["District Type"] == "County Office of Education (COE)" or "charter" in district.lower():
          # print("District: ", county, ', ', district)
          # print("---------------------")
          # for key, value in district_info.items():
          #   if key != 'CDS Code':
          #     print(f"{key}: {value}")
          # print("\n")

          row = [county, district] + np.repeat("N/A", 18).tolist()
          masterTable.loc[len(masterTable.index)] = row
          clear_output()
          display(masterTable)
          continue


        #Get number of schools in a district
        districtDriver = webdriver.Chrome(options=options)
        districtDriver.get(url)
        button = districtDriver.find_element(By.ID, "entity-list-btn")
        button.click()
        table = districtDriver.find_element(By.ID, "DataTables_Table_0")
        tableBody = table.find_element(By.TAG_NAME, "tbody")
        schoolElements = tableBody.find_elements(By.TAG_NAME, "tr")
        district_info["Number of Schools (2021-2022)"] = len(schoolElements)

        #Get Total # of Jurisdictions Per School District and Jurisdiction Name
        if len(schoolElements) == 0:
          district_info["Total # of Jurisdictions Per School District"] = "Error, please manually check"
          district_info["Jurisdiction Name"] = []
        elif schoolElements[0].find_elements(By.TAG_NAME, "td")[0].text == "": #No matching records found
          district_info["Total # of Jurisdictions Per School District"] = "0 - No schools found"
          district_info["Jurisdiction Name"] = []
        else:
          schools = [school.find_elements(By.TAG_NAME, "td")[2] for school in schoolElements]
          jurisdictions = [jurisdiction.text for jurisdiction in schools]
          uniqueJurisdictions = np.unique(jurisdictions)
          district_info["Total # of Jurisdictions Per School District"] = len(uniqueJurisdictions)
          district_info["Jurisdiction Name"] = uniqueJurisdictions

        #Get %unduplicated
        try:
          chart = districtDriver.find_element(By.ID, "chartContent-22")
          div0 = chart.find_elements(By.TAG_NAME, "div")[0]
          div1 = div0.find_element(By.ID, "charttab") #charttab
          div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
          div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
          table = div3.find_elements(By.TAG_NAME, "table")[0]
          tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
          tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
          td = tr.find_elements(By.TAG_NAME, "td")[4]
          span = td.find_elements(By.TAG_NAME, "span")[1]
          district_info["% Unduplicated"] = span.get_attribute("innerHTML")
        except Exception as e:
          district_info["% Unduplicated"] = "Demographic data not available, please manually check"

        #Get % FRM
        try:
          chart = districtDriver.find_element(By.ID, "chartContent-23")
          div0 = chart.find_elements(By.TAG_NAME, "div")[0]
          div1 = div0.find_element(By.ID, "charttab") #charttab
          div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
          div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
          table = div3.find_elements(By.TAG_NAME, "table")[0]
          tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
          tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
          td = tr.find_elements(By.TAG_NAME, "td")[4]
          span = td.find_elements(By.TAG_NAME, "span")[1]
          district_info["% FRM"] = span.get_attribute("innerHTML")
        except Exception as e:
          district_info["% FRM"] = "Demographic data not available, please manually check"

        #Get % English Leaners
        try:
          chart = districtDriver.find_element(By.ID, "chartContent-24")
          div0 = chart.find_elements(By.TAG_NAME, "div")[0]
          div1 = div0.find_element(By.ID, "charttab") #charttab
          div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
          div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
          table = div3.find_elements(By.TAG_NAME, "table")[0]
          tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
          tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
          td = tr.find_elements(By.TAG_NAME, "td")[4]
          span = td.find_elements(By.TAG_NAME, "span")[1]
          district_info["% English Leaners"] = span.get_attribute("innerHTML")
        except Exception as e:
          district_info["% English Leaners"] = "Demographic data not available, please manually check"

        # staffButton = driver.find_element(By.XPATH, '//*[@id="horizontalTab"]/ul/li[3]').click()

        #Get Average Pollution Burden and Average Pollution Burden Percentile
        avgPollutionBurden = pollutionInfoTable[pollutionInfoTable["District"] == district]["Average Pollution Burden"]
        district_info["Average Pollution Burden"] = avgPollutionBurden[0] if len(avgPollutionBurden) > 0 else "N/A"
        avgPollutionBurdenPercentile = pollutionInfoTable[pollutionInfoTable["District"] == district]["Average Pollution Burden Percentile"]
        district_info["Average Pollution Burden Percentile"] = avgPollutionBurdenPercentile[0] if len(avgPollutionBurdenPercentile) > 0 else "N/A"
        district_info["2021-22 Enrollment"] = "N/A"

        row = [county, district, district_info["District Type"], district_info["Grade Span"], district_info["Number of Schools (2021-2022)"],
               "N/A", district_info["Total # of Jurisdictions Per School District"], district_info["Jurisdiction Name"], district_info["2021-22 Enrollment"],
               "# Certified Teachers", "Expense of Education Per ADA", district_info["% Unduplicated"], district_info["% FRM"], district_info["% English Leaners"],
               "N/A", "N/A", "N/A", "N/A", district_info["Average Pollution Burden"], district_info["Average Pollution Burden Percentile"]]
        masterTable.loc[len(masterTable.index)] = row
        clear_output()
        display(masterTable)

        # Print the collected information
    #     print("District: ", county, ', ', district)
    #     print("---------------------")
    #     for key, value in district_info.items():
    #       if key != 'CDS Code':
    #         print(f"{key}: {value}")
    #     print("\n")
    # else:
    #     print('Failed to retrieve data. Status code:', response.status_code)

    # option_elements = [option for option in soup.find_all('option') if option.parent.name == 'select']

,County,District Name,District Type,Grade Levels,Number of Schools\n(2021-22),High School Partner District if Elementary,Total # of Jurisdictions Per School District,Jurisdiction Name\n(list on separate line for each jurisdiction),Student Enrollment \n(2021-22),# of Certificated Teachers\n(2018-19),Expense of Education per ADA \n(2020-21),% Unduplicated \n(2021-22),% FRM \n(2021-22),% English Learners \n(2021-22),Total GO Bonds and Parcel Taxes Attempted \n(2000 - 2029),Total GO Bonds and Parcel Taxes Passed \n(2000 - 2029),Most Recent Passed GO Bond Measure Year\n(2000-2029),Total Amount of GO Bond Measure Funding ($)\n(2000-2029),Average \nPollution Burden Score,Average \nPollution Burden Percentile
0,Alameda,Alameda County Office of Education,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,Alameda,Alameda Unified,Unified School District,K-12,20,N/A,2,"[Alameda, Oakland]",N/A,# Certified Teachers,Expense of Education Per ADA,23.89 %,26 %,10.4 %,N/A,N/A,N/A,N/A,41.659515,54.61
2,Alameda,Albany City Unified,Unified School District,K-12,6,N/A,1,[Albany],N/A,# Certified Teachers,Expense of Education Per ADA,33.75 %,26.7 %,12.8 %,N/A,N/A,N/A,N/A,44.7628,63.84
3,Alameda,Berkeley Unified,Unified School District,K-12,18,N/A,1,[Berkeley],N/A,# Certified Teachers,Expense of Education Per ADA,32.43 %,30 %,7 %,N/A,N/A,N/A,N/A,36.0521,38.75
4,Alameda,Castro Valley Unified,Unified School District,K-12,16,N/A,1,[Castro Valley],N/A,# Certified Teachers,Expense of Education Per ADA,28.99 %,20.5 %,12.7 %,N/A,N/A,N/A,N/A,26.0014,9.69
5,Alameda,Dublin Unified,Unified School District,K-12,15,N/A,1,[Dublin],N/A,# Certified Teachers,Expense of Education Per ADA,13.52 %,8 %,7.1 %,N/A,N/A,N/A,N/A,36.4291,40.22
6,Alameda,Emery Unified,Unified School District,K-12,3,N/A,1,[Emeryville],N/A,# Certified Teachers,Expense of Education Per ADA,88.51 %,62.5 %,19.1 %,N/A,N/A,N/A,N/A,51.9022,81.92
7,Alameda,Fremont Unified,Unified School District,K-12,44,N/A,1,[Fremont],N/A,# Certified Teachers,Expense of Education Per ADA,26.59 %,18.2 %,14.2 %,N/A,N/A,N/A,N/A,34.7598,33.39
8,Alameda,Hayward Unified,Unified School District,K-12,33,N/A,2,"[Castro Valley, Hayward]",N/A,# Certified Teachers,Expense of Education Per ADA,58.12 %,54.5 %,29.3 %,N/A,N/A,N/A,N/A,35.4639,36.44
9,Alameda,Livermore Valley Joint Unified,Unified School District,K-12,19,N/A,1,[Livermore],N/A,# Certified Teachers,Expense of Education Per ADA,25.06 %,20.2 %,10.2 %,N/A,N/A,N/A,N/A,38.0118,45.02


KeyboardInterrupt: ignored

In [ ]:
#Get # of Certified Teachers (2018-2019)
        #---
        # navigationBar = districtDriver.find_element(By.ID, "horizontalTab")
        # navigationBarUl = navigationBar.find_elements(By.TAG_NAME, "ul")[0]
        # li0 = navigationBarUl.find_elements(By.TAG_NAME, "li")[0]
        # li = navigationBarUl.find_elements(By.TAG_NAME, "li")[1]
        #---
        # staffButton.click()
        # viewTableDateButton = districtDriver.find_element(By.ID, "datatable-165")
        # viewTableDateButton.click()
        # table = districtDriver.find_elements(By.CLASS_NAME, "table footable table-bordered master-footable")[0]
        # tableBody = table.find_elements(By.TAG_NAME, "tbody")
        # table = districtDriver.find_elements(By.CLASS_NAME, "chart-data-row total-line")[0]
        # numberWrapper = table.find_elements(By.TAG_NAME, "td")[2]
        # number = numberWrapper.find_elements(By.TAG_NAME, "span")[0].text
        # district_info["# of Certificated Teachers (2018-19)"] = number
        #---
        # districtDriver.execute_script("arguments[0].setAttribute('class', 'tab-label resp-tab-item hor_1')", li0)
        # districtDriver.execute_script("arguments[0].setAttribute('class', 'tab-label resp-tab-item hor_1 resp-tab-active')", li)
        # div = districtDriver.find_element(By.ID, "packeryContainer-11")
        # districtDriver.execute_script("arguments[0].setAttribute('style.display', 'block')", div)
        #---

        #---
        # staffTab = districtDriver.find_element(By.ID, "staff-data-tab")
        # div0 = staffTab.find_elements(By.TAG_NAME, "div")[0] #accordion-content
        # section = div0.find_elements(By.TAG_NAME, "section")[0]
        #---

        # div = section.find_elements(By.TAG_NAME, "div")[0]

        #---
        # div1 = section.find_elements(By.TAG_NAME, "div")[0] #packeryContainer-11
        # print(div1.get_attribute("id"))
        # print(div1.find_elements(By.XPATH, ".//*"))
        # div = div1.find_elements(By.TAG_NAME, "div") #packeryContainer-11
        # print(div)
        # div2 = div.find_elements(By.TAG_NAME, "div")[0] #chart-image
        # div3 = div2.find_elements(By.TAG_NAME, "div")[1] #chartContent
        # div4 = div3.find_elements(By.TAG_NAME, "div")[0] #
        #---

        # chart = districtDriver.find_element(By.ID, "chartContent-165")
        # div0 = chart.find_elements(By.TAG_NAME, "div")[0]

        #---
        # div1 = div4.find_element(By.ID, "charttab") #charttab
        # div2 = div1.find_elements(By.TAG_NAME, "div")[0] #chartdatamenu
        # div3 = div2.find_elements(By.TAG_NAME, "div")[0] #table-div
        # table = div3.find_elements(By.TAG_NAME, "table")[0]
        # tbody = table.find_elements(By.TAG_NAME, "tbody")[0]
        # tr = tbody.find_elements(By.TAG_NAME, "tr")[0]
        # td = tr.find_elements(By.TAG_NAME, "td")[2]
        # span = td.find_elements(By.TAG_NAME, "span")[1]
        # district_info["# of Certificated Teachers (2018-19)"] = span.get_attribute("innerHTML")
        #---

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver



# Variables for the website URL
county = 'Alameda'

countyUrl = f'https://www.ed-data.org/county/{county}'
districts = []
countyResponse = requests.get(countyUrl)

# Check if the request was successful
if countyResponse.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(countyResponse.content, 'html.parser')

    # Find the div element with class "profiledata-container key-att"
    #div_element = soup.find('select', class_='chosen-select-deselect cds').findAll("option")
    # div_element = soup.find('select', {"id":"districtddl"}).find_all("option")
    # div_element = soup.find('div', class_="col-md-4", id="cdsdistrict")

    # div_element = soup.find('div', class_="col-md-4", id="cdsdistrict")
    # inner_element = div_element.find_all("select")[0]
    # # print(div_element)
    # # print(inner_element.find_all("option"))
    # option_elements = [select for select in soup.find_all("td")]
    # # option_elements = inner_element.find_all("option")
    # print(option_elements)
    # for option in option_elements[1:]:
    #   district = option.text.strip()
    #   districts.append(district)

    # div_element = soup.find('select', class_='chosen-select-deselect cds')
    #this gets the districts
    option_elements = [option for option in soup.find_all('option') if option.parent.name == 'select']
    for option in option_elements[1:]:
      district = option.text.strip()
      districts.append(district)

    # Find the ul elements within the div_element
    # div_element = soup.find('div', class_='chosen-drop')
    # district_elements = div_element.find('ul', class_="chosen-results")
    # for li in district_elements:
    #   district = li.text.strip()
    #   districts.append(district)

    # Define a dictionary to store the information
print(districts)

district = 'Hayward-Unified'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'

# Send a GET request to the website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div element with class "profiledata-container key-att"
    div_element = soup.find('div', class_='profiledata-container key-att')

    # Find the ul elements within the div_element
    ul_elements = div_element.find_all('ul', class_='profiledata')

    # Define a dictionary to store the information
    district_info = {}

    # Loop through the ul elements to collect the information
    for ul in ul_elements:
        li_elements = ul.find_all('li')
        if len(li_elements) == 2:
            key = li_elements[0].text.strip()
            value = li_elements[1].find('span').text.strip()
            district_info[key] = value

    # Print the collected information
    print("District: ", county, ', ', district)
    print("---------------------")
    for key, value in district_info.items():
      if key != 'CDS Code':
        print(f"{key}: {value}")
else:
    print('Failed to retrieve data. Status code:', response.status_code)


## Getting Demographic Data from district

read the couple comments below to get the data first

In [ ]:
# Go to the LEA Level CALPADS UPC data in the google sheet -> File ->download as csv
# run this to select the downloaded CALPADS UPC data csv
from google.colab import files


uploaded = files.upload()

In [ ]:
#replace the name of the file with whatever it is named on your computer... then just run the rest to get the values!!
import pandas as pd
import io

UPC_df = pd.read_csv(io.BytesIO(uploaded['calpadsUPC20212022 (4).csv']))

In [ ]:
UPC_df.replace(',','', regex=True, inplace=True)
data_types_dict = {'Free &\nReduced\nMeal\nProgram': int,
                   'Total \nEnrollment': int,
                   'CALPADS Unduplicated \nPupil Count\n(UPC)': int,
                   'English \nLearner \n(EL)': int}


UPC_df = UPC_df.astype(data_types_dict)
UPC_df = UPC_df.loc[:, ['County Name', 'District Name', 'Free &\nReduced\nMeal\nProgram', 'Total \nEnrollment', 'CALPADS Unduplicated \nPupil Count\n(UPC)', 'English \nLearner \n(EL)']]

In [ ]:
def get_district_demographic_info(county, district):
  grouped_by_district = UPC_df.groupby(['County Name', 'District Name']).sum().reset_index()
  our_district = grouped_by_district[(grouped_by_district['County Name'] == county) & (grouped_by_district['District Name'] == district)]
  percent_FRM = float((our_district['Free &\nReduced\nMeal\nProgram'] / our_district['Total \nEnrollment']) *100)
  percent_UPC = float((our_district['CALPADS Unduplicated \nPupil Count\n(UPC)'] / our_district['Total \nEnrollment']) *100)
  percent_EngLearners = float((our_district['English \nLearner \n(EL)'] / our_district['Total \nEnrollment']) *100)
  print("%Unduplicated : " + str(percent_UPC) + "\n %FRM : " +str(percent_FRM) + "\n %English Learners : " + str(percent_EngLearners))

In [ ]:
get_district_demographic_info('Sonoma', 'Bellevue Union')

In [ ]:
# Variables for the website URL

county = 'Sonoma'
district = 'Bellevue Union'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'

# Send a GET request to the website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    headers = {"X-Requested-With": "XMLHttpRequest", 'Accept': 'text/html'}

    # URL to the html table with the General Revenue data. This URL is dynamically requested by the District website
    api_url = "http://www.ed-data.org/Chart/Generate/49-70615-0000000/2018/2022/157?drillinfilter="

    # GET the district website, and GET the dynamically embedded HTML General Revenue table from that
    with requests.Session() as s:
      soup = BeautifulSoup(s.get(url).content, "html.parser")
      soup2 = BeautifulSoup(s.get(api_url, headers=headers).text, "html.parser")
      df = pd.read_html(s.get(api_url, headers=headers).text)
      #print(df)
      df = pd.DataFrame(df[0])
      revenue = df['2020-21'][2]

    print("District: ", county, ', ', district)
    print("---------------------")
    print(f"Expense of Education per ADA 2020-2021: {revenue}")


else:
   print('Failed to retrieve data. Status code:', response.status_code)

# Difficult because of the fact that this value is dynamically loaded by the website, meaning it appears when clicked on.
# Basically,  The table is not always a part of the website like the profile data is , which makes it slightly more annoying,
# but i feel like im getting close

In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
import time

county = 'Sonoma'
district = 'Bellevue Union'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'


service = ChromeService(executable_path='C:/Users/nathan/Downloads/chromedriver_mac64/chromedriver')


driver = webdriver.Chrome(service=service)
# initiating the webdriver. Parameter includes the path of the webdriver.
#driver = webdriver.Chrome('C:/Users/nathan/Downloads/chromedriver_mac64/chromedriver')
driver.get(url)

# this is just to ensure that the page is loaded
time.sleep(5)

html = driver.page_source

# this renders the JS code and stores all
# of the information in static HTML code.

# Now, we could simply apply bs4 to html variable
soup = BeautifulSoup(html, "html.parser")




In [ ]:
pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

# instantiate options
options = webdriver.ChromeOptions()

# run browser in headless mode
options.headless = True

options.binary_location = "C:/Macintosh HD/Applications/Google Chrome/chrome.exe"
# instantiate driver
driver = webdriver.Chrome(service=ChromeService(
	ChromeDriverManager().install()), options=options)

county = 'Sonoma'
district = 'Bellevue Union'

# URL of the Ed-Data website for the specified district
url = f'https://www.ed-data.org/district/{county}/{district}'


# get the entire website content
driver.get(url)

time.sleep(5)

html = driver.page_source

# this renders the JS code and stores all
# of the information in static HTML code.

# Now, we could simply apply bs4 to html variable
soup = BeautifulSoup(html, "html.parser")